In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

trainingSet = pd.read_csv("./data/train.csv")
testingSet = pd.read_csv("./data/test.csv")


In [2]:
def sentiment_analysis(text):
# Extract sentiment of the review text'''
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [ ]:
import pandas as pd

def process(df):
    # This is where you can do all your processing

    df['Helpfulness'] = df['HelpfulnessNumerator'] / df['HelpfulnessDenominator']
    df['Helpfulness'] = df['Helpfulness'].fillna(0)

    df['ReviewLength'] = df.apply(lambda row : len(row['Text'].split()) if type(row['Text']) == str else 0, axis = 1)
    df['Text_Sentiments'] = df['Text'].astype(str).apply(sentiment_analysis)
    df['Summary_Sentiments'] = df['Summary'].astype(str).apply(sentiment_analysis)

    return df

# Load the dataset
trainingSet = pd.read_csv("./data/train.csv")

# Implementing tf-idf vectorizer 
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Process the DataFrame
train_processed = process(trainingSet)

# Load test set
submissionSet = pd.read_csv("./data/test.csv")

# Merge on Id so that the test set can have feature columns as well
testX= pd.merge(train_processed, submissionSet, left_on='Id', right_on='Id')
testX = testX.drop(columns=['Score_x'])
testX = testX.rename(columns={'Score_y': 'Score'})

# The training set is where the score is not null
trainX =  train_processed[train_processed['Score'].notnull()]

# Save the datasets with the new features for easy access later
testX.to_csv("./data/X_test.csv", index=False)
trainX.to_csv("./data/X_train.csv", index=False)


In [13]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# Load training set with new features into DataFrame
X_train = pd.read_csv("./data/X_train.csv")

# Split training set into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(
        X_train.drop(['Score'], axis=1),
        X_train['Score'],
        test_size=1/4.0,
        random_state=0
    )

# This is where you can do more feature selection
X_train_processed = X_train.drop(columns=['Id', 'ProductId', 'UserId', 'Text', 'Summary', 'Time'])
X_test_processed = X_test.drop(columns=['Id', 'ProductId', 'UserId', 'Text', 'Summary', 'Time'])

# model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8)
# model.fit(X_train_processed, Y_train)

model = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=8, subsample=0.7, colsample_bytree=0.8)
model.fit(X_train_processed, Y_train, early_stopping_rounds=10, eval_set=[(X_test_processed, Y_test)])


# pickle model - saves it so you can load it later
with open('xgboost_model.obj', 'wb') as f:
    pickle.dump(model, f)
# to load pickled model: 
# with open('filename', 'rb') as f:
#    model = pickle.load(f)

# Evaluate your model on the testing set
Y_test_predictions = model.predict(X_test_processed)
# print("Accuracy on testing set = ", accuracy_score(Y_test, Y_test_predictions))
print("RMSE on testing set = ", mean_squared_error(Y_test, Y_test_predictions, squared=False))


# Plot a confusion matrix
# cm = confusion_matrix(Y_test, Y_test_predictions, normalize='true')
# sns.heatmap(cm, annot=True)
# plt.title('Confusion matrix of the classifier')
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.show()

[0]	validation_0-rmse:3.77948
[1]	validation_0-rmse:3.74424
[2]	validation_0-rmse:3.70938
[3]	validation_0-rmse:3.67496
[4]	validation_0-rmse:3.64080


/Users/revathivipinachandran/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[5]	validation_0-rmse:3.60685
[6]	validation_0-rmse:3.57354
[7]	validation_0-rmse:3.54044
[8]	validation_0-rmse:3.50755
[9]	validation_0-rmse:3.47540
[10]	validation_0-rmse:3.44358
[11]	validation_0-rmse:3.41202
[12]	validation_0-rmse:3.38078
[13]	validation_0-rmse:3.34990
[14]	validation_0-rmse:3.31965
[15]	validation_0-rmse:3.28936
[16]	validation_0-rmse:3.25946
[17]	validation_0-rmse:3.22983
[18]	validation_0-rmse:3.20053
[19]	validation_0-rmse:3.17153
[20]	validation_0-rmse:3.14286
[21]	validation_0-rmse:3.11449
[22]	validation_0-rmse:3.08673
[23]	validation_0-rmse:3.05886
[24]	validation_0-rmse:3.03143
[25]	validation_0-rmse:3.00417
[26]	validation_0-rmse:2.97731
[27]	validation_0-rmse:2.95061
[28]	validation_0-rmse:2.92432
[29]	validation_0-rmse:2.89815
[30]	validation_0-rmse:2.87243
[31]	validation_0-rmse:2.84688
[32]	validation_0-rmse:2.82176
[33]	validation_0-rmse:2.79690
[34]	validation_0-rmse:2.77230
[35]	validation_0-rmse:2.74783
[36]	validation_0-rmse:2.72376
[37]	validati

In [4]:
X_submission = pd.read_csv("./data/X_test.csv")
X_submission_processed = X_submission.drop(columns=['Id', 'ProductId', 'UserId', 'Text', 'Summary', 'Score', 'Time'])

X_submission['Score'] = model.predict(X_submission_processed)
submission = X_submission[['Id', 'Score']]
submission.to_csv("./data/submission.csv", index=False)